In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import translators as ts
import undetected_chromedriver as uc
import pandas as pd
from selenium import webdriver
import requests
import json
import time
import asyncio
from pymongo import MongoClient
from datetime import datetime
import csv
from google_trans_new import google_translator
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0 
from collections import Counter
from bson.objectid import ObjectId
import unicodedata
import urllib.parse
from difflib import SequenceMatcher

In [2]:
client = MongoClient('mongodb://192.168.1.11:27017/')

# Selecciona la base de datos
db = client['recopilarnovelas']
coleccion_app_novela = db['app_novela']
coleccion_app_capitulo = db['app_capitulo']

In [3]:
def traducir(texto):
    contenido_p = ''
    while True:
        try:
            contenido_p = ts.translate_text(
                texto, translator='bing', to_language='es')
            break
        except Exception as e:
            print(e)
            try:
                contenido_p = ts.translate_text(
                    texto, translator='google', to_language='es')
                break
            except Exception as e:
                print(e)
                try:
                    translator = google_translator() 
                    contenido_p = translator.translate(texto, lang_tgt='es')
                except Exception as e:
                    print(e)
                    pass
                pass
            pass
    return contenido_p

In [24]:
# try:
#     df_list_novels = pd.read_csv('daily_novel_bin_novelbin.csv', sep=';', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
# except:
#     df_list_novels = pd.DataFrame(columns=['nombre', 'url'])
# df_list_novels
df_list_novels = pd.DataFrame(columns=['nombre', 'url'])
# df_existing_novels =pd.read_csv('existing_novels.csv', encoding='utf-8-sig', sep=';', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
# df_existing_novels

In [5]:
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')

In [6]:
url = 'https://novel-bin.net/sort/daily-novel-bin'
# url='https://novel-bin.net/sort/hot-novel-bin'
# url = 'https://novel-bin.net/sort/completed-novel-bin'
# url='https://novel-bin.net/sort/popular-novel-bin'

In [7]:
driver = uc.Chrome(options=options, service = Service(executable_path=ChromeDriverManager().install()))

In [23]:
driver.get(url)
# driver.maximize_window()

In [13]:
(int(driver.find_element(By.XPATH,'/html/body/div/main/div[3]/div/ul/li[9]/a').get_attribute('href').split('=')[-1])+1)

312

In [25]:
if df_list_novels.empty:
#     time.sleep(3)
    list_novels = {"nombre":[],"url":[]}
    page = 2
    limit = (int(driver.find_element(By.XPATH,'/html/body/div/main/div[3]/div/ul/li[9]/a').get_attribute('href').split('=')[-1])+1)
    while page < limit:
        html = driver.page_source
        soup = bs(html, 'html.parser')
        list_novels_page = soup.find_all('h3', class_='novel-title')
        for a_list in list_novels_page:
            list_novels['nombre'].append(a_list.find('a').getText())
            list_novels['url'].append(a_list.find('a').get('href'))
#             print(a_list.find('a').getText(),a_list.find('a').get('href'))
#         time.sleep(1)
        driver.get(f"{url}?page={page}")
        page+= 1
        print(f'{page}, ', end='')
#         break
    df_list_novels = pd.DataFrame(list_novels)

3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223,

In [26]:
df_list_novels.to_csv('daily_novel_bin_novelbin.csv', index=False, encoding='utf-8-sig', sep=';', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
df_list_novels

,nombre,url
0,"Steel, Guns, and the Industrial Party in Anoth...",https://novel-bin.net/novel-bin/steel-guns-and...
1,I Play Yu-Gi-Oh! Card Game,https://novel-bin.net/novel-bin/i-play-yu-gi-o...
2,Supreme Swordsman of the Nine Heavens,https://novel-bin.net/novel-bin/supreme-swords...
3,Reincarnation Of The Strongest Sword God,https://novel-bin.net/novel-bin/reincarnation-...
4,Medical Master,https://novel-bin.net/novel-bin/medical-master
...,...,...
6175,Apocalypse Hunter,https://novel-bin.net/novel-bin/apocalypse-hunter
6176,I Raised Cinderella Preciously,https://novel-bin.net/novel-bin/i-raised-cinde...
6177,Since I’ve Reincarnated as the Villainess’ Fat...,https://novel-bin.net/novel-bin/since-ive-rein...
6178,Wild Malicious Consort: Good For Nothing Ninth...,https://novel-bin.net/novel-bin/wild-malicious...


In [27]:
def get_novel_data(driver, novel_url):
    driver.get(novel_url)
    time.sleep(1)
    html_novel = driver.page_source
    soup_novel = bs(html_novel, 'html.parser')

    title = soup_novel.find('h3', class_='title').getText()
    table_info = soup_novel.find('ul', class_='info info-meta').find_all('li')
    author = ''
    genre = ''
    status = ''
    for li in table_info:
        if 'Author:' in li.find('h3').getText().strip().rstrip():
            author = li.find('a').getText().strip().rstrip()
        if 'Genre:' in li.find('h3').getText().strip().rstrip():
            genre = ', '.join([x.getText().strip().rstrip() for x in li.find_all('a')])
        if 'Status:' in li.find('h3').getText().strip().rstrip():
            status = li.find('a').getText().strip().rstrip()

    img_src = soup_novel.find('div', class_='book').find('img').get('src')
    description = soup_novel.find('div', class_='desc-text').getText().strip().rstrip()

    novel_data = {
        "sitio_id": "65e7e1707963e189aa4c1256",
        "nombre": title.upper(),
        "sinopsis": description,
        "autor": author,
        "genero": genre,
        "status": 'emision' if status == 'Ongoing' else 'completo',
        "url": novel_url,
        "imagen_url": img_src,
        "created_at": datetime.now(),
        "updated_at": datetime.now()
    }

    return novel_data

In [53]:
def get_chapter_data(driver, url_novel, novel_id, collection):
    driver.get(url_novel)
    driver.find_element(By.XPATH, '/html/body/div/main/div[2]/div[1]/div[4]/ul/li[2]/a').click()
    time.sleep(1)
    html_novel = driver.page_source
    soup_novel = bs(html_novel, 'html.parser')
    panel_chapter = soup_novel.find('div', class_='panel-body').find_all('div', class_='row')

#     existing_chapters = {x['url'] for x in collection.find({'novela_id': novel_id}, {'url': 1})}
    existing_chapters = {str(x['nombre']).strip().rstrip() for x in collection.find({'novela_id': novel_id}, {'nombre': 1})}

    for row in panel_chapter:
        lists_chapters = row.find_all('ul', class_='list-chapter')
        for ul in lists_chapters:
            a_list = ul.find_all('a')
            for a in a_list:
                chapter_url = str(a.get('href'))
                if str(a.getText()).strip().rstrip() not in existing_chapters:
                    chapter_data = {
                        "novela_id": novel_id,
                        "nombre": str(a.getText()).strip().rstrip(),
                        "url": chapter_url,
                        "created_at": datetime.now(),
                        "updated_at": datetime.now()
                    }
                    collection.insert_one(chapter_data)
#                 else:
#                     print(f"{chapter_url} si estaba en la base de datos.")

In [51]:
# existing_novels =  [{'_id': str(x['_id']),'nombre': x['nombre'], 'url': x['url']} for x in coleccion_app_novela.find({}, {'_id': 1, 'nombre':1, 'url': 1})]
# print(existing_novels[:5])

In [54]:
existing_novels =  [{'_id': str(x['_id']),'nombre': x['nombre'], 'url': x['url']} for x in coleccion_app_novela.find({}, {'_id': 1, 'nombre':1, 'url': 1})]
existing_novels_names = {x['nombre']:x['_id'] for x in existing_novels}
# print(existing_novels_names.values() if "MARTIAL KING'S RETIRED LIFE" in existing_novels_names.keys() else existing_novels_names)
for idx, novel in df_list_novels.iterrows():

    if str(novel['nombre']).upper().strip().rstrip() in existing_novels_names.keys():
        novel_id = existing_novels_names[str(novel['nombre']).upper().strip().rstrip()]
        print(f"{idx} {novel_id} {str(novel['nombre']).upper().strip().rstrip()} {novel['url']} ya existe, hay que actualizar la URL")
        resultado = coleccion_app_novela.update_one(
            {'_id': ObjectId(novel_id)},
            {'$set': {'url': novel['url']}}
        )
        print(f'Documentos actualizados: {resultado.modified_count}')
        try:
            get_chapter_data(driver, novel['url'], novel_id, coleccion_app_capitulo)
            print(f"Todos los capítulos publicados de '{str(novel['nombre']).upper().strip().rstrip()}' han sido ingresados a la Base de Datos.")
        except Exception as e:
            print(f"Error al procesar la novela '{novel['url']}': {e}")
    else:
        print(f"{str(novel['nombre']).upper().strip().rstrip()} {novel['url']} no existe, hay que crear la novela")
        try:
            novel_data = get_novel_data(driver, novel['url'])
        except:
            driver.get(novel['url'])
            time.sleep(1)
            novel_data = get_novel_data(driver, novel['url'])
        novel_id = str(coleccion_app_novela.insert_one(novel_data).inserted_id)
        print(f"Novela '{novel_data['nombre']}' ingresada a la Base de Datos.")
        try:
            get_chapter_data(driver, novel['url'], novel_id, coleccion_app_capitulo)
            print(f"Todos los capítulos publicados de '{str(novel['nombre']).upper().strip().rstrip()}' han sido ingresados a la Base de Datos.")
        except Exception as e:
            print(f"Error al procesar la novela '{novel['url']}': {e}")
#     break

#     parsed_url1 = urllib.parse.urlparse(novel['url'])
#     path1 = parsed_url1.path.split('/')[-1]

#     found_match = False
#     for novela in existing_novels:
#         parsed_url2 = urllib.parse.urlparse(novela['url'])
#         path2 = parsed_url2.path.split('/')[-1]

#         similarity = SequenceMatcher(None, path1, path2).ratio()

#         if similarity >= 0.52:
#     if not found_match:

0 65f57af373460769e5259bde STEEL, GUNS, AND THE INDUSTRIAL PARTY IN ANOTHER WORLD https://novel-bin.net/novel-bin/steel-guns-and-the-industrial-party-in-another-world ya existe, hay que actualizar la URL
Documentos actualizados: 0
Todos los capítulos publicados de 'STEEL, GUNS, AND THE INDUSTRIAL PARTY IN ANOTHER WORLD' han sido ingresados a la Base de Datos.
1 65f6801f73460769e5284ef3 I PLAY YU-GI-OH! CARD GAME https://novel-bin.net/novel-bin/i-play-yu-gi-oh-card-game ya existe, hay que actualizar la URL
Documentos actualizados: 0
Todos los capítulos publicados de 'I PLAY YU-GI-OH! CARD GAME' han sido ingresados a la Base de Datos.
2 65f574a873460769e5258504 SUPREME SWORDSMAN OF THE NINE HEAVENS https://novel-bin.net/novel-bin/supreme-swordsman-of-the-nine-heavens ya existe, hay que actualizar la URL
Documentos actualizados: 0
Todos los capítulos publicados de 'SUPREME SWORDSMAN OF THE NINE HEAVENS' han sido ingresados a la Base de Datos.
3 65f6b8a873460769e528f1e1 REINCARNATION OF TH

Todos los capítulos publicados de 'MARK OF THE FOOL' han sido ingresados a la Base de Datos.
28 65f78cd973460769e52b4fd7 REBORN AS A TRANSCENDENT https://novel-bin.net/novel-bin/reborn-as-a-transcendent ya existe, hay que actualizar la URL
Documentos actualizados: 0
Todos los capítulos publicados de 'REBORN AS A TRANSCENDENT' han sido ingresados a la Base de Datos.
29 665a8098c5df59f7137262d7 FALLING IN LOVE ONLINE WITH AN ANCIENT GENERAL https://novel-bin.net/novel-bin/falling-in-love-online-with-an-ancient-general ya existe, hay que actualizar la URL
Documentos actualizados: 0
Todos los capítulos publicados de 'FALLING IN LOVE ONLINE WITH AN ANCIENT GENERAL' han sido ingresados a la Base de Datos.
30 65f4d4dd73460769e5239788 RE: MONARCH https://novel-bin.net/novel-bin/re-monarch ya existe, hay que actualizar la URL
Documentos actualizados: 0
Todos los capítulos publicados de 'RE: MONARCH' han sido ingresados a la Base de Datos.
31 65f6644773460769e527fefa DUNGEON DIVER: STEALING A MO

Todos los capítulos publicados de 'I REGRESSED AND THE GENRE CHANGED' han sido ingresados a la Base de Datos.
55 65f92b83346da79f968e60fe ACADEMY’S GENIUS SWORDMASTER https://novel-bin.net/novel-bin/academys-genius-swordmaster ya existe, hay que actualizar la URL
Documentos actualizados: 0
Todos los capítulos publicados de 'ACADEMY’S GENIUS SWORDMASTER' han sido ingresados a la Base de Datos.
56 665a82e5c5df59f713726ad0 DAMN NECROMANCER https://novel-bin.net/novel-bin/damn-necromancer ya existe, hay que actualizar la URL
Documentos actualizados: 0
Todos los capítulos publicados de 'DAMN NECROMANCER' han sido ingresados a la Base de Datos.
THE TALES OF AN INFINITE REGRESSOR https://novel-bin.net/novel-bin/the-tales-of-an-infinite-regressor no existe, hay que crear la novela
Novela 'THE TALES OF AN INFINITE REGRESSOR' ingresada a la Base de Datos.
Todos los capítulos publicados de 'THE TALES OF AN INFINITE REGRESSOR' han sido ingresados a la Base de Datos.
SURVIVING AS A BARBARIAN IN A F

Novela 'THE VILLAIN’S LITTLE SISTER IS SPOILED BY THE MALE LEAD' ingresada a la Base de Datos.
Todos los capítulos publicados de 'THE VILLAIN’S LITTLE SISTER IS SPOILED BY THE MALE LEAD' han sido ingresados a la Base de Datos.
83 65f4f0bb73460769e523e4e2 MYTH BEYOND HEAVEN https://novel-bin.net/novel-bin/myth-beyond-heaven-14570563 ya existe, hay que actualizar la URL
Documentos actualizados: 1
Todos los capítulos publicados de 'MYTH BEYOND HEAVEN' han sido ingresados a la Base de Datos.
84 65faf337ed4d03621f6e8cb9 THE MARTIAL GOD WHO REGRESSED BACK TO LEVEL 2 https://novel-bin.net/novel-bin/the-martial-god-who-regressed-back-to-level-2 ya existe, hay que actualizar la URL
Documentos actualizados: 1
Todos los capítulos publicados de 'THE MARTIAL GOD WHO REGRESSED BACK TO LEVEL 2' han sido ingresados a la Base de Datos.
MYSTERIES OF IMMORTAL PUPPET MASTER https://novel-bin.net/novel-bin/mysteries-of-immortal-puppet-master no existe, hay que crear la novela
Novela 'MYSTERIES OF IMMORTAL 

Todos los capítulos publicados de 'I BECAME A FLASHING GENIUS AT THE MAGIC ACADEMY' han sido ingresados a la Base de Datos.
111 6642a6a62576936696d2cda4 IMMORTAL PATH SPACE https://novel-bin.net/novel-bin/immortal-path-space ya existe, hay que actualizar la URL
Documentos actualizados: 0
Todos los capítulos publicados de 'IMMORTAL PATH SPACE' han sido ingresados a la Base de Datos.
112 65f5186d73460769e524552f CELESTIAL BLOODLINE https://novel-bin.net/novel-bin/celestial-bloodline ya existe, hay que actualizar la URL
Documentos actualizados: 1
Todos los capítulos publicados de 'CELESTIAL BLOODLINE' han sido ingresados a la Base de Datos.
113 65f5027073460769e524162b IMMORTAL OF THE AGES https://novel-bin.net/novel-bin/immortal-of-the-ages ya existe, hay que actualizar la URL
Documentos actualizados: 1
Todos los capítulos publicados de 'IMMORTAL OF THE AGES' han sido ingresados a la Base de Datos.
114 65f54b5173460769e524fd8e VILLAIN: MANIPULATING THE HEROINES INTO HATING THE PROTAGONIS

...

Todos los capítulos publicados de 'THE STORY OF BLOOD AND ROSES' han sido ingresados a la Base de Datos.
6041 65fa86d1ed4d03621f6b99fc THE LUST SYSTEM https://novel-bin.net/novel-bin/the-lust-system ya existe, hay que actualizar la URL
Documentos actualizados: 1
Todos los capítulos publicados de 'THE LUST SYSTEM' han sido ingresados a la Base de Datos.
6042 65fa86d6ed4d03621f6b9ac1 ASSASSINATION SYSTEM https://novel-bin.net/novel-bin/assassination-system ya existe, hay que actualizar la URL
Documentos actualizados: 1
Todos los capítulos publicados de 'ASSASSINATION SYSTEM' han sido ingresados a la Base de Datos.
6043 65fa86dbed4d03621f6b9bd5 MY MCV AND DOOMSDAY https://novel-bin.net/novel-bin/my-mcv-and-doomsday ya existe, hay que actualizar la URL
Documentos actualizados: 1
Todos los capítulos publicados de 'MY MCV AND DOOMSDAY' han sido ingresados a la Base de Datos.
6044 65fa86e1ed4d03621f6b9e88 I HAVE BECAME ABLE TO DO ANYTHING WITH MY GROWTH CHEAT BUT I CAN NOT SEEM TO GET OUT OF 

Todos los capítulos publicados de 'MUSHOKU TENSEI (WN)' han sido ingresados a la Base de Datos.
6070 65fa877aed4d03621f6bc8ae RE:MONSTER https://novel-bin.net/novel-bin/remonster ya existe, hay que actualizar la URL
Documentos actualizados: 1
Todos los capítulos publicados de 'RE:MONSTER' han sido ingresados a la Base de Datos.
6071 65fa877eed4d03621f6bc8d9 THE INVERTED DRAGON'S SCALE https://novel-bin.net/novel-bin/the-inverted-dragons-scale ya existe, hay que actualizar la URL
Documentos actualizados: 1
Todos los capítulos publicados de 'THE INVERTED DRAGON'S SCALE' han sido ingresados a la Base de Datos.
6072 65fa8784ed4d03621f6bcaed WEAPON MASTER https://novel-bin.net/novel-bin/weapon-master ya existe, hay que actualizar la URL
Documentos actualizados: 1
Todos los capítulos publicados de 'WEAPON MASTER' han sido ingresados a la Base de Datos.
6073 65fa878eed4d03621f6bd3d2 SPIRIT HUNTERS OF MAOSHAN SECT https://novel-bin.net/novel-bin/spirit-hunters-of-maoshan-sect ya existe, hay qu

Todos los capítulos publicados de 'LOVE WITH DANGER IN THE PURPLE: BEING BELOVED BY THE HARDHEARTED AVENGER' han sido ingresados a la Base de Datos.
6099 65fa8816ed4d03621f6bef87 PAMPERED FEI BRIMMING WITH CUTENESS https://novel-bin.net/novel-bin/pampered-fei-brimming-with-cuteness ya existe, hay que actualizar la URL
Documentos actualizados: 1
Todos los capítulos publicados de 'PAMPERED FEI BRIMMING WITH CUTENESS' han sido ingresados a la Base de Datos.
6100 65fa881aed4d03621f6bf08a SHADOW ROGUE https://novel-bin.net/novel-bin/shadow-rogue ya existe, hay que actualizar la URL
Documentos actualizados: 1
Todos los capítulos publicados de 'SHADOW ROGUE' han sido ingresados a la Base de Datos.
6101 65fa881fed4d03621f6bf119 I AM REALLY A SUPERSTAR https://novel-bin.net/novel-bin/i-am-really-a-superstar ya existe, hay que actualizar la URL
Documentos actualizados: 1
Todos los capítulos publicados de 'I AM REALLY A SUPERSTAR' han sido ingresados a la Base de Datos.
6102 65fa8828ed4d03621f6bf

Todos los capítulos publicados de 'THE FEAST' han sido ingresados a la Base de Datos.
6128 65fa88b6ed4d03621f6c1f41 NAZE BOKU NO SEKAI WO DARE MO OBOETEINAINOKA? https://novel-bin.net/novel-bin/naze-boku-no-sekai-wo-dare-mo-oboeteinainoka ya existe, hay que actualizar la URL
Documentos actualizados: 1
Todos los capítulos publicados de 'NAZE BOKU NO SEKAI WO DARE MO OBOETEINAINOKA?' han sido ingresados a la Base de Datos.
6129 65fa88bbed4d03621f6c1f60 CHU WANG FEI https://novel-bin.net/novel-bin/chu-wang-fei ya existe, hay que actualizar la URL
Documentos actualizados: 1
Todos los capítulos publicados de 'CHU WANG FEI' han sido ingresados a la Base de Datos.
6130 65fa88c0ed4d03621f6c2042 PICKED UP BY THE PROTAGONIST OF A TORMENTED! MC NOVEL https://novel-bin.net/novel-bin/picked-up-by-the-protagonist-of-a-tormented-mc-novel ya existe, hay que actualizar la URL
Documentos actualizados: 1
Todos los capítulos publicados de 'PICKED UP BY THE PROTAGONIST OF A TORMENTED! MC NOVEL' han sido in

Todos los capítulos publicados de 'TI SHEN' han sido ingresados a la Base de Datos.
6155 65fa893aed4d03621f6c2ed5 PATH TO HEAVEN https://novel-bin.net/novel-bin/path-to-heaven ya existe, hay que actualizar la URL
Documentos actualizados: 1
Todos los capítulos publicados de 'PATH TO HEAVEN' han sido ingresados a la Base de Datos.
6156 65fa893fed4d03621f6c3016 MAGIC BULLET IN MAGIC LAND https://novel-bin.net/novel-bin/magic-bullet-in-magic-land ya existe, hay que actualizar la URL
Documentos actualizados: 1
Todos los capítulos publicados de 'MAGIC BULLET IN MAGIC LAND' han sido ingresados a la Base de Datos.
6157 65fa8944ed4d03621f6c302c COLD KING'S DOMINATING LOVE, GENIUS MEDICAL CONSORT https://novel-bin.net/novel-bin/cold-kings-dominating-love-genius-medical-consort ya existe, hay que actualizar la URL
Documentos actualizados: 1
Todos los capítulos publicados de 'COLD KING'S DOMINATING LOVE, GENIUS MEDICAL CONSORT' han sido ingresados a la Base de Datos.
6158 65fa8948ed4d03621f6c309a 

In [55]:
df_existing_novels = pd.DataFrame(existing_novels)
df_existing_novels

,_id,nombre,url
0,65e7ed3d9297a4c7c5aaaeef,MARTIAL KING'S RETIRED LIFE,https://novel-bin.net/novel-bin/martial-kings-...
1,65e874957a145405d0b24bf5,CULTIVATION CHAT GROUP,https://novel-bin.net/novel-bin/cultivation-ch...
2,65efbb2ceb6e829adc4a104a,EMPEROR'S DOMINATION,https://novel-bin.net/novel-bin/emperors-domin...
3,65efbb58eb6e829adc4a104c,BECOMING IMMORTAL THROUGH GETTING MARRIED,https://novel-bin.net/novel-bin/becoming-immor...
4,65f1af5d4285ef9618f6b28f,"AS AN IMMORTAL, I ONLY LEARN FORBIDDEN SKILLS",https://novel-bin.net/novel-bin/as-an-immortal...
...,...,...,...
6187,665a8297c5df59f713726a5c,OVERPOWERED IN THE WRONG GENRE,https://novel-bin.net/novel-bin/overpowered-in...
6188,665a82a3c5df59f713726a73,SAND MAGE OF THE BURNT DESERT,https://novel-bin.net/novel-bin/sand-mage-of-t...
6189,665a82aec5df59f713726a92,THE SUPPORTING CHARACTERS IN THE GAME HAVE BEC...,https://novel-bin.net/novel-bin/the-supporting...
6190,665a82bfc5df59f713726ab1,I REGRESSED AND THE GENRE CHANGED,https://novel-bin.net/novel-bin/i-regressed-an...


In [56]:
df_existing_novels.to_csv('existing_novels.csv', index=False, encoding='utf-8-sig', sep=';', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)

In [57]:
driver.quit()